<a href="https://colab.research.google.com/github/dakilaledesma/NCBG/blob/main/tools/Multilabel_Parts_Minthresh_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q /content/drive/MyDrive/UNC/NCBG/sept1_classif_training.zip -d /content
! cp /content/drive/MyDrive/UNC/NCBG/qry_fsus_trainingdata_ai.xlsx /content/qry_fsus_trainingdata_ai.xlsx

CPU times: user 54.3 ms, sys: 21.9 ms, total: 76.2 ms
Wall time: 7.87 s


In [2]:
! mv /content/content/images /content/images
! rm -rf /content/content

In [10]:
! cp -r drive/MyDrive/UNC/NCBG/mp_1b/ ./mp_1b

In [3]:
! pip install -U tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.0 MB 14 kB/s 
     |████████████████████████████████| 438 kB 66.8 MB/s 
     |████████████████████████████████| 1.7 MB 53.6 MB/s 
     |████████████████████████████████| 5.9 MB 54.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5

In [4]:
! apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 30 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 10s (41.5 MB/s)
(Reading database ... 155685 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 155663 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+

In [5]:
! pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 8.9 MB/s 


In [6]:
import pandas as pd

df = pd.read_excel("qry_fsus_trainingdata_ai.xlsx")

In [7]:
class_dict = {'aspect': 0, 'bud': 1, 'fall-color': 2, 'flower': 3, 'fruit': 4, 'leaf': 5, 'leaf-underside': 6, 'other': 7, 'root': 8, 'seed': 9, 'side-by-side': 10, 'stem': 11, 'venation': 12, 'vestiture': 13, 'whole-plant': 14}

In [8]:
import numpy as np

im_to_class = {}
for idx, row in df.iterrows():
  anno_bn = row["file"]
  try:
    classes = row["image_context"].split(';')
  except AttributeError:
    continue
  
  out_anno = np.zeros(15, dtype=int)
  for class_ in classes:
    class_idx = class_dict.get(class_)
    out_anno[class_idx] = 1
  
  im_to_class[anno_bn] = out_anno

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.applications.convnext import ConvNeXtBase
from tensorflow.keras.layers import Dense
import tensorflow_addons as tfa
import keras

opt = tfa.optimizers.AdaBelief(learning_rate=1e-3)
custom_objects = {"AdaBelief": opt}

with keras.utils.custom_object_scope(custom_objects):
  model = load_model("mp_1b")

In [14]:
from glob import glob
import os
from PIL import Image, UnidentifiedImageError
from tqdm.notebook import tqdm

preds = []
y = []
for im_fn in tqdm(list(glob("images/*.*"))):
  bn = os.path.basename(im_fn).split("__")[1].replace(".jpg", '')
  bn = f"{bn}.jpg"
  try:
    y.append(im_to_class[bn])
    im = Image.open(im_fn)
    im = im.resize((331, 331))
    im = np.array(im, dtype=int)
    im = np.expand_dims(im, axis=0)

    preds.append(list(np.array(model(im))[0]))
  except KeyError or UnidentifiedImageError:
    continue

  0%|          | 0/8613 [00:00<?, ?it/s]

In [15]:
print(preds[:5])

[[5.1582436e-08, 4.409734e-06, 1.9920118e-07, 7.895201e-05, 8.140399e-09, 1.0936035e-06, 4.207569e-09, 1.1640221e-09, 1.9866513e-05, 1.3217162e-06, 2.7556883e-09, 0.99999857, 3.421227e-08, 4.8329557e-08, 9.375024e-10], [1.31357365e-05, 3.6056236e-09, 3.213406e-05, 5.5127117e-07, 5.0746243e-05, 8.89007e-07, 0.999998, 1.3869524e-06, 3.3633236e-07, 2.3416262e-06, 4.2798998e-08, 0.00010810365, 3.8012786e-07, 1.078066e-05, 5.3706217e-09], [1.0442485e-10, 3.322233e-06, 1.4269605e-08, 1.944592e-06, 7.9763754e-07, 0.99999607, 2.1774492e-08, 1.6641421e-07, 1.2615281e-07, 4.5471418e-09, 8.777656e-10, 5.2250773e-05, 1.0993644e-05, 0.00013261496, 3.6997795e-07], [0.9999887, 2.980726e-07, 1.0128919e-05, 0.9999945, 1.9767922e-06, 3.4184888e-11, 2.137698e-07, 4.805944e-06, 5.8782665e-05, 6.374307e-05, 2.4950012e-08, 0.0003214779, 3.0778288e-10, 1.7081746e-07, 0.00011302618], [0.9999877, 3.7845034e-09, 1.2857774e-08, 2.8658664e-07, 1.5666958e-05, 2.3638462e-05, 2.665556e-09, 1.644687e-07, 1.1264529e-0

In [16]:
from sklearn.metrics import accuracy_score

In [19]:
for thresh in range(1, 10):
  thresh = thresh / 10
  thresh_pred = [[1 if v > thresh else 0 for v in l] for l in preds]
  print(thresh, accuracy_score(y, thresh_pred))

0.1 0.9512310048297797
0.2 0.966780539521734
0.3 0.9730239132995642
0.4 0.9770290964777948
0.5 0.9784426905406998
0.6 0.9804452821298151
0.7 0.9819766756979621
0.8 0.9806808811402992
0.9 0.977382494993521
